# Apresentação do notebook  

**Do projeto:**  Uma Investigação sobre a Evasão Escolar no Ensino Médio Brasileiro durante a Pandemia de Covid-19

**Objetivo:**  Criação dos DataFrames de trabalho e tratamento dos dados brutos para a EDA (dados do ano-base de 2020)  


# Preparação dos dados  




## Montagem do Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Data Girls/Grace Hopper/entrega_final

## Criação dos DataFrames e tratamento dos dados

### » Microdados do Censo Escolar

#### Carregamento dos dados  

> Premissas:   
> - Abaixo serão incluídas apenas as variáveis determinadas na documentação do projeto
> - Serão analisadas apenas as escolas que não tenham sido extintas durante o perído analisado, ou seja, aquelas com as seguintes condições: `TP_SITUACAO_FUNCIONAMENTO == 1` ou `TP_SITUACAO_FUNCIONAMENTO == 2`
> - Como o objetivo é analisar o impacto da pandemia na evasão escolar no Ensino Médio, serão mantidas apenas escolas que possuam ao menos uma matrícula nesse período, ou seja, em que `QT_MAT_MED > 0`


In [ ]:
import pandas as pd
import numpy as np

filepath = os.path.join(os.getcwd(),"data","raw","microdados_2019.zip")

parametros = [
    'NU_ANO_CENSO','NO_REGIAO','CO_REGIAO','SG_UF','CO_UF','NO_MUNICIPIO','CO_MUNICIPIO','CO_ENTIDADE','NO_ENTIDADE',
    'TP_DEPENDENCIA','TP_LOCALIZACAO','TP_SITUACAO_FUNCIONAMENTO', 'IN_AGUA_POTAVEL','IN_ENERGIA_REDE_PUBLICA',
    'IN_ENERGIA_GERADOR_FOSSIL','IN_ENERGIA_RENOVAVEL', 'IN_ESGOTO_REDE_PUBLICA','IN_ESGOTO_FOSSA','IN_LIXO_SERVICO_COLETA',
    'IN_LIXO_DESTINO_FINAL_PUBLICO', 'IN_TRATAMENTO_LIXO_INEXISTENTE','IN_BANHEIRO','IN_BANHEIRO_PNE',
    'IN_BIBLIOTECA_SALA_LEITURA', 'IN_LABORATORIO_CIENCIAS','IN_LABORATORIO_INFORMATICA','IN_ACESSIBILIDADE_CORRIMAO',
    'IN_ACESSIBILIDADE_ELEVADOR', 'IN_ACESSIBILIDADE_PISOS_TATEIS','IN_ACESSIBILIDADE_VAO_LIVRE','IN_ACESSIBILIDADE_RAMPAS',
    'IN_ACESSIBILIDADE_SINAL_SONORO','IN_ACESSIBILIDADE_SINAL_TATIL','IN_ACESSIBILIDADE_SINAL_VISUAL',
    'IN_ACESSIBILIDADE_INEXISTENTE','QT_DESKTOP_ALUNO','QT_COMP_PORTATIL_ALUNO','QT_TABLET_ALUNO','IN_INTERNET',
    'IN_ALIMENTACAO','IN_MATERIAL_PED_MULTIMIDIA','IN_MATERIAL_PED_CIENTIFICO','IN_MATERIAL_PED_DIFUSAO',
    'IN_MATERIAL_PED_MUSICAL','IN_MATERIAL_PED_JOGOS','IN_MATERIAL_PED_ARTISTICAS','IN_MATERIAL_PED_DESPORTIVA',
    'IN_MATERIAL_PED_CAMPO','IN_MEDIACAO_PRESENCIAL','IN_MEDIACAO_SEMIPRESENCIAL','IN_MEDIACAO_EAD','IN_DIURNO',
    'IN_NOTURNO','IN_MED','QT_MAT_BAS','QT_MAT_MED','QT_MAT_BAS_FEM','QT_MAT_BAS_MASC','QT_MAT_BAS_ND',
    'QT_MAT_BAS_BRANCA','QT_MAT_BAS_PRETA','QT_MAT_BAS_PARDA','QT_MAT_BAS_AMARELA','QT_MAT_BAS_INDIGENA',
    'QT_DOC_MED','QT_TUR_MED', 'TP_LOCALIZACAO_DIFERENCIADA'
]

# Leitura do arquivo em chunks
microdados_iter = pd.read_csv(
  filepath, sep=";", header=0, index_col='CO_ENTIDADE', encoding='latin1',
  usecols=parametros, chunksize=10000  # Lê 10.000 linhas por vez
)

# Filtra os dados requeridos durante a leitura e concatena os pedaços
microdados_2020 = pd.concat([
    chunk[(chunk['TP_SITUACAO_FUNCIONAMENTO'].isin([1, 2])) & (chunk['QT_MAT_MED'] > 0)]
    for chunk in microdados_iter
])

#### Inspeção do DataFrame

In [ ]:
validador_nulos = (
    microdados_2020
    .isna()
    .sum()
    .to_frame('Valores Nulos')
    .assign(Percentual=lambda df: (df['Valores Nulos'] / len(microdados_2020) * 100).round(2))
    .sort_values(by='Valores Nulos', ascending=False)
)
print(validador_nulos[validador_nulos['Valores Nulos'] > 0])

Empty DataFrame
Columns: [Valores Nulos, Percentual]
Index: []


Obs.: Será necessário tratar coluna `QT_DESKTOP_ALUNO`,
`QT_COMP_PORTATIL_ALUNO`e `QT_TABLET_ALUNO` (todas vaiáveis de tipo int64), pois apresentam o valor 88888. Segundo o dicionário de dados disponibilizado pelo INEP, este valor representa valores extremos observados no dataset. Demais valores foram verificados e estão ok.  

#### Pré-tratamento dos dados

**Referente às variáveis `IN_MATERIAL_PED_MULTIMIDIA `,  `IN_MATERIAL_PED_CIENTIFICO `, `IN_MATERIAL_PED_DIFUSAO`,  `IN_MATERIAL_PED_MUSICAL `,  `IN_MATERIAL_PED_JOGOS `,
     `IN_MATERIAL_PED_ARTISTICAS `,  `IN_MATERIAL_PED_DESPORTIVA `,  `IN_MATERIAL_PED_CAMPO`:**

> Justificativa:
> -  Estas variáveis contêm valores não informados registrados na base, sendo isso representado pelo número 9 na base original.
> - Para esta análise iremos assumir que o valor 'não informado' é equivalente a valor "false"/"não" e, portanto, vamos atribuir o número 0 para representar isso.

In [ ]:
cols_material = [
  'IN_MATERIAL_PED_MULTIMIDIA', 'IN_MATERIAL_PED_CIENTIFICO',
  'IN_MATERIAL_PED_DIFUSAO', 'IN_MATERIAL_PED_MUSICAL', 'IN_MATERIAL_PED_JOGOS',
  'IN_MATERIAL_PED_ARTISTICAS', 'IN_MATERIAL_PED_DESPORTIVA', 'IN_MATERIAL_PED_CAMPO'
]

microdados_2020[cols_material] = microdados_2020[cols_material].replace(9, 0)

**Referente às variáveis `QT_DESKTOP_ALUNO`, `QT_COMP_PORTATIL_ALUNO`e `QT_TABLET_ALUNO`:**
> Justificativa:
> - Na base original, foi atribuído pelo INEP o valor 88888 (tipo de dado int64) para representar outliers ou dados inválidos para as três variáveis acima.
> Calculamos a proporção destes valores para estas três variáveis.  
> - Diante do baixíssimo impacto no dataset (menor que 5%), foi aplicado o seguinte tratamento: atribuição de um valor nulo 'intencional' (NA) para que tais valores não sejam considerados durante a fase seguinte da Análise Exploratória dos Dados



In [ ]:
cols_dispositivos = ['QT_DESKTOP_ALUNO', 'QT_COMP_PORTATIL_ALUNO', 'QT_TABLET_ALUNO']

# Calcula o número total de registros
total_registros = len(microdados_2020)

# Calcula o número de ocorrências de 88888 por coluna
ocorrencias_88888 = (microdados_2020[cols_dispositivos] == 88888).sum()

# Calcula a proporção
proporcao_88888 = (ocorrencias_88888 / total_registros)

print("Proporção de valores 88888 por coluna:")
print(f'{proporcao_88888}')

Proporção de valores 88888 por coluna:
QT_DESKTOP_ALUNO          0.000933
QT_COMP_PORTATIL_ALUNO    0.000069
QT_TABLET_ALUNO           0.000069
dtype: float64


In [ ]:
# Substitui os valores 88888 por pd.NA
microdados_2020[cols_dispositivos] = microdados_2020[cols_dispositivos].replace(88888, pd.NA)

#### Criação de novas variáveis  



- Criação do índice IN_ENERGIA  
> Indice de energia

In [ ]:
# Condições
condicoes = [
    (microdados_2020['IN_ENERGIA_REDE_PUBLICA'] == 1) |
    (microdados_2020['IN_ENERGIA_GERADOR_FOSSIL'] == 1) |
    (microdados_2020['IN_ENERGIA_RENOVAVEL'] == 1)
]

# Se as condições não forem atendidas, o valor será 0
microdados_2020['IN_ENERGIA'] = np.select(condicoes, [1], default=0)

- Criação do índice IN_ESGOTO  
> Indíce de esgoto

In [ ]:
condicoes = [
    (microdados_2020['IN_ESGOTO_REDE_PUBLICA'] == 1) |
    (microdados_2020['IN_ESGOTO_FOSSA'] == 1)
]

microdados_2020['IN_ESGOTO'] = np.select(condicoes, [1], default=0)

- Criação do índice IN_COLETA_LIXO  
> Ìndice de coleta de lixo
>
>  Observação: Na coluna ```IN_TRATAMENTO_LIXO_INEXISTENTE```, o valor "0" representa que o tratamento de lixo não é inexistente, ou seja, há tratamento de lixo. A variável ```IN_TRATAMENTO_LIXO``` apenas inverte a lógica, de forma que "0" passa a realmente representar a ausência de tratamento de lixo.

In [ ]:
condicoes = [
    (microdados_2020['IN_LIXO_SERVICO_COLETA'] == 1) |
    (microdados_2020['IN_LIXO_DESTINO_FINAL_PUBLICO'] == 1) |
    (microdados_2020['IN_TRATAMENTO_LIXO_INEXISTENTE'] == 0)
]

microdados_2020['IN_COLETA_LIXO'] = np.select(condicoes, [1], default=0)

- Criação do índice 'IND_INFRA_BASICA'  
> Mede a presença de recursos essenciais na escola, como água potável, energia elétrica, esgoto, tratamento de lixo e banheiros (variável numérica discreta, em uma escala de 0 a 5)

In [ ]:
microdados_2020['IND_INFRA_BASICA'] = microdados_2020[[
    'IN_AGUA_POTAVEL',
    'IN_ENERGIA',
    'IN_ESGOTO',
    'IN_COLETA_LIXO',
    'IN_BANHEIRO'
]].sum(axis=1)

- Criação do índice 'IND_ACESSIBILIDADE'  
> Avalia as condições de acessibilidade na escola  para pessoas com deficiência, considerando
itens como banheiros adaptados, corrimãos,
elevadores, rampas, pisos táteis, sinalizações, etc. (variável numérica discreta, em uma escala de 0 a 9)

In [ ]:
microdados_2020['IND_ACESSIBILIDADE'] = microdados_2020[[
    'IN_BANHEIRO_PNE',
    'IN_ACESSIBILIDADE_CORRIMAO',
    'IN_ACESSIBILIDADE_ELEVADOR',
    'IN_ACESSIBILIDADE_PISOS_TATEIS',
    'IN_ACESSIBILIDADE_VAO_LIVRE',
    'IN_ACESSIBILIDADE_RAMPAS',
    'IN_ACESSIBILIDADE_SINAL_SONORO',
    'IN_ACESSIBILIDADE_SINAL_TATIL',
    'IN_ACESSIBILIDADE_SINAL_VISUAL',
    'IN_ACESSIBILIDADE_INEXISTENTE'
]].sum(axis=1)

- Criação do índice 'IND_DIVERSIDADE_MATERIAL_PED'    
> Mede a variedade de materiais
pedagógicos disponíveis na escola, como
multimídia, equipamentos
científicos, instrumentos
musicais, jogos educativos e
materiais esportivos e artísticos.(variável numérica discreta em uma escala de 0 a 8)

In [ ]:
microdados_2020['IND_DIVERSIDADE_MATERIAL_PED'] = microdados_2020[[
    'IN_MATERIAL_PED_MULTIMIDIA',
    'IN_MATERIAL_PED_CIENTIFICO',
    'IN_MATERIAL_PED_DIFUSAO',
    'IN_MATERIAL_PED_MUSICAL',
    'IN_MATERIAL_PED_JOGOS',
    'IN_MATERIAL_PED_ARTISTICAS',
    'IN_MATERIAL_PED_DESPORTIVA',
    'IN_MATERIAL_PED_CAMPO'
]].sum(axis=1)

-  Criação do índice 'IND_INFRA_EDUCACIONAL'  
> Avalia a disponibilidade de
espaços educacionais na escola, como
bibliotecas, laboratórios de
ciências e de informática (variável numérica discreta em uma escala de 0 a 3)



In [ ]:
microdados_2020['IND_INFRA_EDUCACIONAL'] = microdados_2020[[
    'IN_BIBLIOTECA_SALA_LEITURA',
    'IN_LABORATORIO_CIENCIAS',
    'IN_LABORATORIO_INFORMATICA'
]].sum(axis=1)

- Criação do índice 'QT_DISPOSITIVOS_ALUNOS'  
> Número de computadores e/ou dispositivos eletrônicos em uso pelos alunos

In [ ]:
microdados_2020['QT_DISPOSITIVOS_ALUNOS'] = microdados_2020[[
    'QT_DESKTOP_ALUNO',
    'QT_COMP_PORTATIL_ALUNO',
    'QT_TABLET_ALUNO',
]].sum(axis=1)

- Criação do índice de matrículas no Ensino Médio,  categorizados por gênero e cor/raça    

> Abaixo são calculadas as quantidades de matrículas proporcionais no Ensino Médio, com base nos dados das matrículas no Ensino Básico

In [ ]:
colunas = [
    'QT_MAT_BAS_FEM', 'QT_MAT_BAS_MASC', 'QT_MAT_BAS_ND', 'QT_MAT_BAS_BRANCA', 'QT_MAT_BAS_PRETA', 'QT_MAT_BAS_PARDA',
    'QT_MAT_BAS_AMARELA', 'QT_MAT_BAS_INDIGENA'
]

# Cálculo das proporções para o Ensino Médio
for coluna in colunas:
    microdados_2020[f'QT_MAT_MED_{coluna.split("_")[-1]}'] = (
        microdados_2020[coluna].div(microdados_2020['QT_MAT_BAS'].replace(0, 1)) * microdados_2020['QT_MAT_MED']
    ).round()

Exclusão de atributos não necessários
> Dada a criação e armazenamento dos índices acima, os seguintes atributos listados abaixo serão excluídos do DataFrame por não serem mais necessários a partir de agora.  

In [ ]:
microdados_2020 = microdados_2020.drop(columns=[
    'IN_ENERGIA_REDE_PUBLICA','IN_ENERGIA_GERADOR_FOSSIL', 'IN_ENERGIA_RENOVAVEL','IN_ENERGIA', 'IN_ESGOTO_REDE_PUBLICA',
    'IN_ESGOTO_FOSSA','IN_ESGOTO', 'IN_LIXO_SERVICO_COLETA', 'IN_LIXO_DESTINO_FINAL_PUBLICO','IN_TRATAMENTO_LIXO_INEXISTENTE',
    'IN_COLETA_LIXO', 'IN_AGUA_POTAVEL','IN_BANHEIRO', 'IN_BANHEIRO_PNE','IN_ACESSIBILIDADE_CORRIMAO','IN_ACESSIBILIDADE_ELEVADOR',
    'IN_ACESSIBILIDADE_PISOS_TATEIS', 'IN_ACESSIBILIDADE_VAO_LIVRE','IN_ACESSIBILIDADE_RAMPAS','IN_ACESSIBILIDADE_SINAL_SONORO',
    'IN_ACESSIBILIDADE_SINAL_TATIL', 'IN_ACESSIBILIDADE_SINAL_VISUAL','IN_ACESSIBILIDADE_INEXISTENTE', 'IN_MATERIAL_PED_MULTIMIDIA',
    'IN_MATERIAL_PED_CIENTIFICO', 'IN_MATERIAL_PED_DIFUSAO','IN_MATERIAL_PED_MUSICAL','IN_MATERIAL_PED_JOGOS',
    'IN_MATERIAL_PED_ARTISTICAS','IN_MATERIAL_PED_DESPORTIVA', 'IN_MATERIAL_PED_CAMPO', 'IN_BIBLIOTECA_SALA_LEITURA',
    'IN_LABORATORIO_CIENCIAS','IN_LABORATORIO_INFORMATICA', 'QT_DESKTOP_ALUNO','QT_COMP_PORTATIL_ALUNO','QT_TABLET_ALUNO',
    'QT_MAT_BAS','QT_MAT_BAS_FEM', 'QT_MAT_BAS_MASC', 'QT_MAT_BAS_ND','QT_MAT_BAS_BRANCA', 'QT_MAT_BAS_PRETA', 'QT_MAT_BAS_PARDA',
    'QT_MAT_BAS_AMARELA', 'QT_MAT_BAS_INDIGENA'
])

#### Tratamento dos dados

Ajuste de Tipos

In [ ]:
# Casting de int64 para object
codigos = ['CO_REGIAO', 'CO_UF', 'CO_MUNICIPIO']
microdados_2020[codigos] = microdados_2020[codigos].astype(object)

In [ ]:
# Casting de int64 para bool
booleanos = [
    'IN_INTERNET', 'IN_ALIMENTACAO', 'IN_MEDIACAO_PRESENCIAL', 'IN_MEDIACAO_SEMIPRESENCIAL',
    'IN_MEDIACAO_EAD','IN_DIURNO', 'IN_NOTURNO', 'IN_MED'
]
microdados_2020[booleanos] = microdados_2020[booleanos].astype(bool)

In [ ]:
# Casting de int64 float32
num = [
    'QT_DOC_MED','QT_TUR_MED','IND_INFRA_BASICA','IND_ACESSIBILIDADE','IND_DIVERSIDADE_MATERIAL_PED',
    'IND_INFRA_EDUCACIONAL','QT_DISPOSITIVOS_ALUNOS'
]
microdados_2020[num] = microdados_2020[num].astype('float32')

In [ ]:
# Casting de float64 para float32
num = [
    'QT_MAT_MED_FEM','QT_MAT_MED_MASC','QT_MAT_MED_ND','QT_MAT_MED_BRANCA','QT_MAT_MED_PRETA','QT_MAT_MED_PARDA',
    'QT_MAT_MED_AMARELA','QT_MAT_MED_INDIGENA', 'QT_MAT_MED'
]
microdados_2020[num] = microdados_2020[num].astype('float32')

In [ ]:
# Casting de int 64 para category
categoricos = ['TP_DEPENDENCIA', 'TP_LOCALIZACAO', 'TP_SITUACAO_FUNCIONAMENTO', 'TP_LOCALIZACAO_DIFERENCIADA']
microdados_2020[categoricos] = microdados_2020[categoricos].astype('category')

# Renomeia as categorias
microdados_2020['TP_DEPENDENCIA'] = microdados_2020['TP_DEPENDENCIA'].cat.rename_categories({
    1: 'Federal',
    2: 'Estadual',
    3: 'Municipal',
    4: 'Privada'
})

microdados_2020['TP_LOCALIZACAO'] = microdados_2020['TP_LOCALIZACAO'].cat.rename_categories({1: 'Urbana', 2: 'Rural'})

microdados_2020['TP_SITUACAO_FUNCIONAMENTO'] = microdados_2020['TP_SITUACAO_FUNCIONAMENTO'].cat.rename_categories({
    1: 'Em Atividade',
    2: 'Paralisada'
})

microdados_2020['TP_LOCALIZACAO_DIFERENCIADA'] = microdados_2020['TP_LOCALIZACAO_DIFERENCIADA'].cat.rename_categories({
    0: 'Não se aplica',
    1: 'Assentamento',
    2: 'Indígena',
    3: 'Rem. Quilombo'
})

In [ ]:
# Casting de int 64 para uint16
microdados_2020['NU_ANO_CENSO'] = microdados_2020['NU_ANO_CENSO'].astype('uint16')
microdados_2020.rename(columns={'NU_ANO_CENSO':'ANO'}, inplace=True)

Resturação da variável CO_ENTIDADE de microdados_2020 para coluna para o merge que virá adiante

In [ ]:
microdados_2020 = microdados_2020.reset_index()

### » Média de Alunos por Turma

#### Carregamento dos dados

> Após exame da base de dados disponibilizada pelo INEP, foi verificado que ali estão listadas  escolas basileiras que oferecem o Ensino Básico, sendo que ao serem selecionadas as colunas de interesse para este projeto, referentes apenas ao Ensino Médio, escolas que não oferecem esta etapa de ensino são listadas também com seus respectivos campos contendo o valor **`--`** para representar inexistência do Ensino Médio na instituição.  
>
> Portanto, para otimizar o carregamento dos dados, optamos por:  
- Logo na leitura, transformar estes valores inválidos (`--`) em NaN (representando assim a ausência de valor)
- Na sequência, aplicar um filtro para remover linhas em que haja **concomitantemente** valores NaN nos campos de médias de alunos, pois isso significa que possivelmente a escola não ofereceu Ensino Médio durante o período analisado.  
- Observação: Escolas que, por ventura, tenham registrados valores de média referentes a apenas  uma ou duas séries do Ensino Médio também serão normalmente carregadas no DataFrame de trabalho.

In [ ]:
# Caminho do arquivo zipado - Alunos por Turma
zip_path = os.getcwd() + '/data/raw/alunos_turma_2019.zip'
excel_file = 'alunos_turma_2019.xlsx'

with zipfile.ZipFile(zip_path) as z:
    with z.open(excel_file) as f:
        alunos_turma_2020 = pd.read_excel(f, header=8, na_values='--')

# Filtra colunas necessárias
alunos_turma_2020 = alunos_turma_2020[[
    'CO_ENTIDADE', 'NO_ENTIDADE', 'MED_CAT_0', 'MED_01_CAT_0', 'MED_02_CAT_0','MED_03_CAT_0'
]]

print(f'Total de escolas da base de dados (inclui todas da Educação Básica): {len(alunos_turma_2020)}')

Total de escolas da base de dados (inclui todas da Educação Básica): 175341


Seleção das escolas que efetivamente oferecem Ensino Médio

In [ ]:
colunas_med = ['MED_CAT_0', 'MED_01_CAT_0', 'MED_02_CAT_0', 'MED_03_CAT_0']
alunos_turma_2020.dropna(subset=colunas_med, how='all', inplace=True)

print(f'Total de escolas que oferecem Ensino Médio: {len(alunos_turma_2020)}')

Total de escolas que oferecem Ensino Médio: 28908


#### Inspeção do DataFrame

In [ ]:
validador_nulos = (
    alunos_turma_2020
    .isna()
    .sum()
    .to_frame('Valores Nulos')
    .assign(Percentual=lambda df: (df['Valores Nulos'] / len(alunos_turma_2020) * 100).round(2))
    .sort_values(by='Valores Nulos', ascending=False)
)
print(validador_nulos[validador_nulos['Valores Nulos'] > 0])

              Valores Nulos  Percentual
MED_03_CAT_0           1496        5.18
MED_02_CAT_0           1113        3.85
MED_01_CAT_0            667        2.31


Obs: Foi feita a verificação por valores inválidos na base e não há.

#### Tratamento dos dados


Conversão de tipos e novos nomes para colunas


In [ ]:
num = ['MED_CAT_0','MED_01_CAT_0','MED_02_CAT_0','MED_03_CAT_0']

# Converte as colunas selecionadas para float16
alunos_turma_2020[num] = alunos_turma_2020[num].astype('float32')

# Renomeia colunas para nomes mais amigáveis
alunos_turma_2020.rename(columns={
    'MED_CAT_0':'ALUNOS_TURMA_MED',
    'MED_01_CAT_0':'ALUNOS_TURMA_MED1',
    'MED_02_CAT_0':'ALUNOS_TURMA_MED2',
    'MED_03_CAT_0':'ALUNOS_TURMA_MED3'
}, inplace=True)

# Padronização da variável que será chave para merge na sequência (float64 para int64)
alunos_turma_2020['CO_ENTIDADE'] = alunos_turma_2020['CO_ENTIDADE'].astype('int64')

### » Taxa de Rendimento Escolar


#### Carregamento dos dados  

> Após exame da base de dados disponibilizada pelo INEP, foi verificado que ali estão listadas escolas brasileiras que ofereceram Ensino Básico no ano de 2020, sendo que ao serem selecionadas apenas as colunas de interesse para este projeto - aquelas referentes apenas ao Ensino Médio - são listadas também escolas que não ofereceram esta etapa de ensino (com seus respectivos campos contendo o valor `--` para representar a inexistência da Ensino Médio na instituição).  
>
> Portanto, para otimizar o carregamento dos dados, optamos aqui por:  
>
> - Logo na leitura, transformar estes valores inválidos (`--`) em NaN (representando assim a ausência de valor).  
> - Na sequência, aplicar um filtro para remover linhas em que haja **concomitantemente** valores NaN nos campos de taxas de rendimento de alunos, pois isso significa que a escola possivelmente não ofereceu Ensino Médio durante o período analisado.   
> - Observação: Escolas que, por ventura, tenham registrado taxas de rendimento referentes a apenas uma ou duas séries do Ensino Médio continuarão normalmente no DataFrame de trabalho.  

In [ ]:
# Caminho do arquivo zipado - Taxa de Rendimento
zip_path = os.getcwd() + '/data/raw/tx_rendimento_2019.zip'
excel_file = 'tx_rendimento_2019.xlsx'

with zipfile.ZipFile(zip_path) as z:
  with z.open(excel_file) as f:
    # Substitui valores '--' verificados por NaN
    tx_rendimento_2020 = pd.read_excel(f, header=8, na_values='--')

tx_rendimento_2020 = tx_rendimento_2020[[
    'CO_ENTIDADE', 'NO_ENTIDADE', 'tap_MED', 'tap_M01', 'tap_M02', 'tap_M03', 'tre_MED', 'tre_M01', 'tre_M02', 'tre_M03',
    'tab_MED', 'tab_M01', 'tab_M02', 'tab_M03'
]]

print(f'Total de escolas nesta base de dados do INEP (inclui todas da Educação Básica): {len(tx_rendimento_2020)}')

Total de escolas nesta base de dados do INEP (inclui todas da Educação Básica): 131231


Seleção das escolas que efetivamente oferecem Ensino Médio

In [ ]:
colunas_med = [
    'tap_MED', 'tap_M01', 'tap_M02', 'tap_M03', 'tre_MED', 'tre_M01', 'tre_M02',
    'tre_M03', 'tab_MED', 'tab_M01', 'tab_M02', 'tab_M03'
]
tx_rendimento_2020.dropna(subset=colunas_med, how='all', inplace=True)

print(f'Total de escolas que efetivamente ofereceram Ensino Médio no período: {len(tx_rendimento_2020)}')

Total de escolas que efetivamente ofereceram Ensino Médio no período: 28633


#### Inspeção do DataFrame

In [ ]:
validador_nulos = (
    tx_rendimento_2020
    .isna()
    .sum()
    .to_frame('Valores Nulos')
    .assign(Percentual=lambda df: (df['Valores Nulos'] / len(tx_rendimento_2020) * 100).round(2))
    .sort_values(by='Valores Nulos', ascending=False)
)
print(validador_nulos[validador_nulos['Valores Nulos'] > 0])

         Valores Nulos  Percentual
tap_M03           2065        7.21
tre_M03           2065        7.21
tab_M03           2065        7.21
tab_M02           1123        3.92
tre_M02           1123        3.92
tap_M02           1123        3.92
tab_M01            693        2.42
tre_M01            693        2.42
tap_M01            693        2.42


Obs: Foi feita a verificação por valores inválidos na base e não há.

####  Tratamento dos dados

Ajuste de tipos, conversão para decimal e novos nomes para colunas

In [ ]:
num = [
    'tap_MED','tap_M01','tap_M02','tap_M03','tre_MED','tre_M01','tre_M02','tre_M03','tab_MED','tab_M01',
    'tab_M02','tab_M03'
]

# Conversão de taxa percentual para decimal
tx_rendimento_2020[num] = tx_rendimento_2020[num] / 100

# Ajuste de tipos
tx_rendimento_2020[num] = tx_rendimento_2020[num].astype('float32')

# Renomeando colunas
tx_rendimento_2020.rename(columns={
    'tap_MED':'TAP_MED','tap_M01':'TAP_MED1','tap_M02':'TAP_MED2','tap_M03':'TAP_MED3','tre_MED':'TRE_MED',
    'tre_M01':'TRE_MED1','tre_M02':'TRE_MED2','tre_M03':'TRE_MED3','tab_MED':'TAB_MED','tab_M01':'TAB_MED1',
    'tab_M02':'TAB_MED2','tab_M03':'TAB_MED3'
}, inplace=True)

# Padronização da variável que será chave para merge na sequência (float64 para int64)
tx_rendimento_2020['CO_ENTIDADE'] = tx_rendimento_2020['CO_ENTIDADE'].astype('int64')

### » Taxa de Transição (evasão)

#### Carregamento dos dados  

> Após exame da base de dados disponibilizada pelo INEP, foi verificado que ali estão listados  munincípios brasileiros com escolas que oferecem o Ensino Básico - sendo que ao serem selecionadas as colunas de interesse para este projeto, referentes apenas ao Ensino Médio, municípios que não oferecem esta etapa de ensino são listados também com seus respectivos campos contendo o valor **`--`** ou *`***`* para representar inexistência da Ensino Médio.  
>
> Portanto, para otimizar o carregamento dos dados, optou-se aqui por:  
- Logo na leitura, transformar estes valores inválidos (`--` e `***`) em NaN (representando assim a ausência de valor)
- Na sequência, aplicar um filtro para remover linhas em que haja **concomitantemente** valores NaN nos campos de taxas de evasão, pois isso significa que o município possivelmente não ofereceu Ensino Médio durante o ano de 2020.
- Observação: Municípios que, por ventura, tenham registrado taxas referentes a apenas uma ou duas séries do Ensino Médio também serão normalmente carregados no DataFrame de trabalho.

In [ ]:
# Caminho do arquivo zipado - Taxa de Evasão
zip_path = os.getcwd() + '/data/raw/tx_transicao_2019_2020.zip'
excel_file = 'tx_transicao_2019_2020.xlsx'

with zipfile.ZipFile(zip_path) as z:
    with z.open(excel_file) as f:
        tx_transicao_2020 = pd.read_excel(f, header=8, index_col='CO_MUNICIPIO', na_values=['--', '***'])

tx_transicao_2020 = tx_transicao_2020[[
    'NO_LOCALIZACAO', 'NO_DEPENDENCIA', '1_CAT3_CATMED', '1_CAT3_CATMED_01', '1_CAT3_CATMED_02', '1_CAT3_CATMED_03'
]]

# Colunas renomeadas
num = ['1_CAT3_CATMED','1_CAT3_CATMED_01','1_CAT3_CATMED_02','1_CAT3_CATMED_03']
tx_transicao_2020.rename(columns={
    'NO_LOCALIZACAO':'TP_LOCALIZACAO','NO_DEPENDENCIA':'TP_DEPENDENCIA','1_CAT3_CATMED':'TEV_MED',
    '1_CAT3_CATMED_01':'TEV_MED1','1_CAT3_CATMED_02':'TEV_MED2','1_CAT3_CATMED_03':'TEV_MED3'
}, inplace=True)

print(f'Total de linhas da base original do INEP: {len(tx_transicao_2020)}')

Total de linhas da base original do INEP: 23761


Seleção dos municípios que oferecem Ensino Médio

In [ ]:
colunas_med = ['TEV_MED', 'TEV_MED1', 'TEV_MED2', 'TEV_MED3']
tx_transicao_2020.dropna(subset=colunas_med, how='all', inplace=True)

print(f'Total de municípios que efetivamente ofereceram Ensino Médio no período: {len(tx_transicao_2020)}')

Total de municípios que efetivamente ofereceram Ensino Médio no período: 19723


#### Inspeção do DataFrame

In [ ]:
validador_nulos = (
    tx_transicao_2020
    .isna()
    .sum()
    .to_frame('Valores Nulos')
    .assign(Percentual=lambda df: (df['Valores Nulos'] / len(tx_transicao_2020) * 100).round(2))
    .sort_values(by='Valores Nulos', ascending=False)
)
print(validador_nulos[validador_nulos['Valores Nulos'] > 0])

          Valores Nulos  Percentual
TEV_MED3            244        1.24
TEV_MED2            119        0.60
TEV_MED1             68        0.34


Obs: Foi feita a verificação por valores inválidos na base e não há.

#### Tratamento dos dados

Ajuste de tipos e conversão para decimal

In [ ]:
num = ['TEV_MED', 'TEV_MED1', 'TEV_MED2', 'TEV_MED3']

# Transforma porcentagem em número decimal
tx_transicao_2020[num] = tx_transicao_2020[num] / 100

# Converte variáveis selecionadas para tipo float16
tx_transicao_2020[num] = tx_transicao_2020[num].astype('float32')

Resturação da variável CO_MUNICIPIO para coluna preparando o merge que virá adiante

In [ ]:
tx_transicao_2020 = tx_transicao_2020.reset_index()

In [ ]:
# Casting de float64 para int64 para compatibilizar variável para o merge
tx_transicao_2020['CO_MUNICIPIO'] = tx_transicao_2020['CO_MUNICIPIO'].astype('int64')

### » aluno_rendimento_2020  



#### Criação do DataFrame    

> Este DataFrame é originado da junção de dados de outros dois DataFrames: Média de Alunos por Turma (alunos_turma_2020) e Taxa de Rendimento Escolar (tx_rendimento_2020)

In [ ]:
aluno_rendimento_2020 = pd.merge(
  alunos_turma_2020,
  tx_rendimento_2020,
    on=['CO_ENTIDADE', 'NO_ENTIDADE'],
  how='left'
)

In [ ]:
print(f'DF alunos_turma_2020 → {len(alunos_turma_2020):,} registros'.replace(',', '.'))
print(f'DF tx_rendimento_2020 → {len(tx_rendimento_2020):,} registros'.replace(',', '.'))
print(f'Resultado do merge → {len(aluno_rendimento_2020):,} registros'.replace(',', '.'))

DF alunos_turma_2020 → 28.908 registros
DF tx_rendimento_2020 → 28.633 registros
Resultado do merge → 28.908 registros


#### Inspeção do DataFrame

In [ ]:
validador_nulos = (
    aluno_rendimento_2020
    .isna()
    .sum()
    .to_frame('Valores Nulos')
    .assign(Percentual=lambda df: (df['Valores Nulos'] / len(aluno_rendimento_2020) * 100).round(2))
    .sort_values(by='Valores Nulos', ascending=False)
)
print(validador_nulos[validador_nulos['Valores Nulos'] > 0])

                   Valores Nulos  Percentual
TRE_MED3                    2342        8.10
TAP_MED3                    2342        8.10
TAB_MED3                    2342        8.10
ALUNOS_TURMA_MED3           1496        5.18
TRE_MED2                    1401        4.85
TAB_MED2                    1401        4.85
TAP_MED2                    1401        4.85
ALUNOS_TURMA_MED2           1113        3.85
TAB_MED1                     971        3.36
TRE_MED1                     971        3.36
TAP_MED1                     971        3.36
ALUNOS_TURMA_MED1            667        2.31
TRE_MED                      280        0.97
TAP_MED                      280        0.97
TAB_MED                      280        0.97


### » dados_escolas_2020  



#### Criação do DataFrame  

> Este DataFrame é a junção de dados de outros dois DataFrames: Microdados do Censo Escolar (microdados_2020) e aluno_rendimento_2020  


In [ ]:
dados_escolas_2020 = pd.merge(
    microdados_2020,
    aluno_rendimento_2020,
    on=['CO_ENTIDADE', 'NO_ENTIDADE'],
    how='left'
)

In [ ]:
print(f'DF microdados_2020 → {len(microdados_2020):,} registros'.replace(',', '.'))
print(f'DF aluno_rendimento_2020 → {len(aluno_rendimento_2020):,} registros'.replace(',', '.'))
print(f'Resultado do merge → {len(dados_escolas_2020):,} registros'.replace(',', '.'))

DF microdados_2020 → 28.933 registros
DF aluno_rendimento_2020 → 28.908 registros
Resultado do merge → 28.933 registros


#### Tratamento dos dados

Transformação das variáveis categóricas
> As alterações abaixo foram necessárias para garantir a compatibilidade entre este DataFrame e o DataFrame referente à taxa de evasão escolar, que discrimina a coluna TP_DEPENDENCIA apenas entre público e privado. No entanto, para preservar o maior número de informações possíveis, foram criadas novas variáveis booleanas.

In [ ]:
# Variável TP_DEPENDENCIA
dados_escolas_2020['IN_MUNICIPAL'] = (dados_escolas_2020['TP_DEPENDENCIA'] == 'Municipal').astype(bool)
dados_escolas_2020['IN_ESTADUAL'] = (dados_escolas_2020['TP_DEPENDENCIA'] == 'Estadual').astype(bool)
dados_escolas_2020['IN_FEDERAL'] = (dados_escolas_2020['TP_DEPENDENCIA'] == 'Federal').astype(bool)
dados_escolas_2020['IN_PRIVADA'] = (dados_escolas_2020['TP_DEPENDENCIA'] == 'Privada').astype(bool)

# Variável TP_SITUACAO_FUNCIONAMENTO
dados_escolas_2020['IN_ATIVA'] = (dados_escolas_2020['TP_SITUACAO_FUNCIONAMENTO'] == 'Em Atividade').astype(bool)
dados_escolas_2020['IN_PARALISADA'] = (dados_escolas_2020['TP_SITUACAO_FUNCIONAMENTO'] == 'Paralisada').astype(bool)

# Variável TP_LOCALIZACAO_DIFERENCIADA
dados_escolas_2020['AR_NAO_DIFER'] = (dados_escolas_2020['TP_LOCALIZACAO_DIFERENCIADA'] == 'Não se aplica').astype(bool)
dados_escolas_2020['AR_ASSENTAMENTO'] = (dados_escolas_2020['TP_LOCALIZACAO_DIFERENCIADA'] == 'Assentamento').astype(bool)
dados_escolas_2020['AR_INDIGENA'] = (dados_escolas_2020['TP_LOCALIZACAO_DIFERENCIADA'] == 'Indígena').astype(bool)
dados_escolas_2020['AR_REM_QUILOMBO'] = (dados_escolas_2020['TP_LOCALIZACAO_DIFERENCIADA'] == 'Rem. Quilombo').astype(bool)

Substituição de valores

In [ ]:
# Conversão para string
dados_escolas_2020['TP_DEPENDENCIA'] = dados_escolas_2020['TP_DEPENDENCIA'].astype(str)

# Substitui valores
dados_escolas_2020['TP_DEPENDENCIA'] = dados_escolas_2020['TP_DEPENDENCIA'].replace({
    'Federal': 'Pública',
    'Estadual': 'Pública',
    'Municipal': 'Pública'
})

# Conversão de volta para categoria
dados_escolas_2020['TP_DEPENDENCIA'] = dados_escolas_2020['TP_DEPENDENCIA'].astype('category')

dados_escolas_2020['REDE_PUBLICA'] = (dados_escolas_2020['TP_DEPENDENCIA'] == 'Pública').astype(bool)
dados_escolas_2020['REDE_PRIVADA'] = (dados_escolas_2020['TP_DEPENDENCIA'] == 'Privada').astype(bool)

Exclusão de variáveis não necessárias mais

In [ ]:
dados_escolas_2020 = dados_escolas_2020.drop(columns=[
    'TP_DEPENDENCIA', 'TP_LOCALIZACAO_DIFERENCIADA', 'TP_SITUACAO_FUNCIONAMENTO'
])

#### Criação de novas variáveis  

. Variável TIPO_DEPENDENCIA

In [ ]:
def tipo_escola(row):
    if row["IN_PRIVADA"] == 1:
        return "Privada"
    elif row["IN_MUNICIPAL"] == 1 or row["IN_ESTADUAL"] == 1 or row["IN_FEDERAL"] == 1:
        return "Pública"
    else:
        return "Não identificada"

dados_escolas_2020["TIPO_DEPENDENCIA"] = dados_escolas_2020.apply(tipo_escola, axis=1)

. Variáveis razão aluno/docente e razão aluno/turma

In [ ]:
dados_escolas_2020['RAZ_ALUNO_DOC'] = (
    dados_escolas_2020['QT_MAT_MED'] / dados_escolas_2020['QT_DOC_MED']
).where(dados_escolas_2020['QT_DOC_MED'] > 0)

dados_escolas_2020['RAZ_ALUNO_TUR'] = (
    dados_escolas_2020['QT_MAT_MED'] / dados_escolas_2020['QT_TUR_MED']
).where(dados_escolas_2020['QT_TUR_MED'] > 0)

Ajuste de tipos

In [ ]:
num = ['RAZ_ALUNO_DOC', 'RAZ_ALUNO_TUR']

# Ajuste de tipos
dados_escolas_2020[num] = dados_escolas_2020[num].astype('float32')

#### Mapeamento de ausência de valores

In [ ]:
validador_nulos = (
    dados_escolas_2020
    .isna()
    .sum()
    .to_frame('Valores Nulos')
    .assign(Percentual=lambda df: (df['Valores Nulos'] / len(dados_escolas_2020) * 100).round(2))
    .sort_values(by='Valores Nulos', ascending=False)
)
print(validador_nulos[validador_nulos['Valores Nulos'] > 0])

                   Valores Nulos  Percentual
TAP_MED3                    2367        8.18
TRE_MED3                    2367        8.18
TAB_MED3                    2367        8.18
ALUNOS_TURMA_MED3           1521        5.26
TAB_MED2                    1426        4.93
TRE_MED2                    1426        4.93
TAP_MED2                    1426        4.93
ALUNOS_TURMA_MED2           1138        3.93
TRE_MED1                     996        3.44
TAB_MED1                     996        3.44
TAP_MED1                     996        3.44
ALUNOS_TURMA_MED1            692        2.39
TAB_MED                      305        1.05
TRE_MED                      305        1.05
TAP_MED                      305        1.05
ALUNOS_TURMA_MED              25        0.09


In [ ]:
traducao_loc = {
    'AR_NAO_DIFER': 'Não Diferenciada',
    'AR_REM_QUILOMBO': 'Comun. Quilombolas',
    'AR_ASSENTAMENTO': 'Assentamento',
    'AR_INDIGENA': 'Comun. Indígenas'
}

# Lista de colunas de localização diferenciada
loc_dif_cols = list(traducao_loc.keys())

series = {
    'TAP_MED1': '1ª série',
    'TAP_MED2': '2ª série',
    'TAP_MED3': '3ª série'
}

# Lista para armazenar os resultados
tabelas = []

# Loop pelas séries e tipos de localização
for serie_col, serie_nome in series.items():
    for loc_col in loc_dif_cols:
        # Filtra escolas daquela localização
        df_filtrado = dados_escolas_2020[dados_escolas_2020[loc_col] == 1]
        total = df_filtrado.shape[0]

        # Conta quantas escolas NÃO ofertam a série (baseado em TAP_MED*.isna())
        sem_oferta = df_filtrado[serie_col].isna().sum()

        # Calcula proporção
        prop = sem_oferta / total if total > 0 else None

        # Adiciona ao resultado
        tabelas.append({
            'Série': serie_nome,
            'Tipo_Localizacao': traducao_loc[loc_col],
            'Total de Escolas': total,
            'Sem essa série': int(sem_oferta),
            'Proporção (%)': round(prop * 100, 2) if prop is not None else None
        })

resultado = pd.DataFrame(tabelas)
resultado

,Série,Tipo_Localizacao,Total de Escolas,Sem essa série,Proporção (%)
0,1ª série,Não Diferenciada,28095,892,3.17
1,1ª série,Comun. Quilombolas,112,4,3.57
2,1ª série,Assentamento,240,14,5.83
3,1ª série,Comun. Indígenas,486,86,17.70
4,2ª série,Não Diferenciada,28095,1286,4.58
5,2ª série,Comun. Quilombolas,112,5,4.46
6,2ª série,Assentamento,240,26,10.83
7,2ª série,Comun. Indígenas,486,109,22.43
8,3ª série,Não Diferenciada,28095,2200,7.83
9,3ª série,Comun. Quilombolas,112,21,18.75


In [ ]:
series = {
    'TAP_MED1': '1ª série',
    'TAP_MED2': '2ª série',
    'TAP_MED3': '3ª série'
}

# Lista de resultados
resultados_urbanorural = []

# Loop por série e tipo de localização
for serie_col, serie_nome in series.items():
    for local in ['Urbana', 'Rural']:
        # Filtra escolas pela localização
        df_local = dados_escolas_2020[dados_escolas_2020['TP_LOCALIZACAO'] == local]
        total = df_local.shape[0]

        # Conta quantas escolas não oferecem a série (TAP_MED*.isna())
        sem_oferta = df_local[serie_col].isna().sum()
        prop = sem_oferta / total if total > 0 else None

        resultados_urbanorural.append({
            'Série': serie_nome,
            'Localização': local,
            'Total de Escolas': total,
            'Sem essa série': int(sem_oferta),
            'Proporção (%)': round(prop * 100, 2) if prop is not None else None
        })

df_urbanorural = pd.DataFrame(resultados_urbanorural)
print(df_urbanorural)

      Série Localização  Total de Escolas  Sem essa série  Proporção (%)
0  1ª série      Urbana             25838             742           2.87
1  1ª série       Rural              3095             254           8.21
2  2ª série      Urbana             25838            1100           4.26
3  2ª série       Rural              3095             326          10.53
4  3ª série      Urbana             25838            1916           7.42
5  3ª série       Rural              3095             451          14.57


**Conclusão:**

Os valores ausentes identificados nas variáveis de rendimento escolar por série correspondem majoritariamente a escolas em zonas rurais ou contextos específicos (indígenas, assentamentos, quilombolas), onde determinadas séries do Ensino Médio não são ofertadas. Assim, os NaN representam ausência legítima de oferta, e não inconsistência na base.

Verificação de eventuais registros duplicados

In [ ]:
duplicatas_completas = dados_escolas_2020.duplicated()
print(f"Quantidade de linhas duplicadas completas: {duplicatas_completas.sum()}")

Quantidade de linhas duplicadas completas: 0


In [ ]:
# Verifica se existem registros duplicados de escolas com o mesmo CO_ENTIDADE
duplicatas_escolas = dados_escolas_2020.duplicated(subset=['CO_ENTIDADE'])
print(f"Quantidade de linhas duplicadas por escola: {duplicatas_escolas.sum()}")

Quantidade de linhas duplicadas por escola: 0


#### Salvando a base para análises posteriores

In [ ]:
dados_escolas_2020.to_csv('dados_escolas_2020.csv', encoding='latin1', index=True)

### » dados_municipios_2020  




#### Criação do DataFrame  

> Este DataFrame é originado do agrupamento de variáveis e cálculo de agregação de valores do DataFrame dados_escolas_2020

In [ ]:
# Cria função para cálculo de média ponderada
def media_ponderada(col, peso):
    def func(x):
        pesos = dados_escolas_2020.loc[x.index, peso]
        valores = x

        # Filtra pares válidos (ambos não nulos)
        validos = ~(valores.isna() | pesos.isna())
        pesos_validos = pesos[validos]
        valores_validos = valores[validos]

        soma_pesos = pesos_validos.sum()
        if soma_pesos == 0:
            return np.nan
        return (valores_validos * pesos_validos).sum() / soma_pesos
    return func

# Dicinário de agregação
agg_dict = {
    'ANO': 'first',
    'NO_REGIAO': 'first',
    'CO_REGIAO': 'first',
    'SG_UF': 'first',
    'CO_UF': 'first',
    'NO_MUNICIPIO': 'first',

    'IN_INTERNET': 'mean',
    'IN_ALIMENTACAO': 'mean',
    'IN_MEDIACAO_PRESENCIAL': 'mean',
    'IN_MEDIACAO_SEMIPRESENCIAL': 'mean',
    'IN_MEDIACAO_EAD': 'mean',
    'IN_DIURNO': 'mean',
    'IN_NOTURNO': 'mean',

    'IN_MED': 'sum',
    'QT_MAT_MED': 'sum',
    'QT_DOC_MED': 'sum',
    'QT_TUR_MED': 'sum',

    'IND_INFRA_BASICA': media_ponderada('IND_INFRA_BASICA', 'QT_MAT_MED'),
    'IND_ACESSIBILIDADE': media_ponderada('IND_ACESSIBILIDADE', 'QT_MAT_MED'),
    'IND_DIVERSIDADE_MATERIAL_PED': media_ponderada('IND_DIVERSIDADE_MATERIAL_PED', 'QT_MAT_MED'),
    'IND_INFRA_EDUCACIONAL': media_ponderada('IND_INFRA_EDUCACIONAL', 'QT_MAT_MED'),

    'QT_DISPOSITIVOS_ALUNOS': 'sum',

    'QT_MAT_MED_FEM': 'sum',
    'QT_MAT_MED_MASC': 'sum',
    'QT_MAT_MED_ND': 'sum',
    'QT_MAT_MED_BRANCA': 'sum',
    'QT_MAT_MED_PRETA': 'sum',
    'QT_MAT_MED_PARDA': 'sum',
    'QT_MAT_MED_AMARELA': 'sum',
    'QT_MAT_MED_INDIGENA': 'sum',

    'ALUNOS_TURMA_MED': 'mean',
    'ALUNOS_TURMA_MED1': 'mean',
    'ALUNOS_TURMA_MED2': 'mean',
    'ALUNOS_TURMA_MED3': 'mean',

    'TAP_MED': media_ponderada('TAP_MED', 'QT_MAT_MED'),
    'TAP_MED1': 'mean',
    'TAP_MED2': 'mean',
    'TAP_MED3': 'mean',

    'TRE_MED': media_ponderada('TRE_MED', 'QT_MAT_MED'),
    'TRE_MED1': 'mean',
    'TRE_MED2': 'mean',
    'TRE_MED3': 'mean',

    'TAB_MED': media_ponderada('TAB_MED', 'QT_MAT_MED'),
    'TAB_MED1': 'mean',
    'TAB_MED2': 'mean',
    'TAB_MED3': 'mean',

    'IN_MUNICIPAL': 'sum',
    'IN_ESTADUAL': 'sum',
    'IN_FEDERAL': 'sum',
    'IN_PRIVADA': 'sum',
    'IN_ATIVA': 'sum',
    'IN_PARALISADA': 'sum',
    'REDE_PUBLICA': 'sum',
    'REDE_PRIVADA': 'sum',
    'AR_NAO_DIFER': 'sum',
    'AR_ASSENTAMENTO': 'sum',
    'AR_INDIGENA': 'sum',
    'AR_REM_QUILOMBO': 'sum',

    'RAZ_ALUNO_DOC': media_ponderada('RAZ_ALUNO_DOC', 'QT_MAT_MED'),
    'RAZ_ALUNO_TUR': media_ponderada('RAZ_ALUNO_TUR', 'QT_MAT_MED')
}

# Roda groupby() e agregação
dados_municipios_2020 = dados_escolas_2020.groupby(
    ['CO_MUNICIPIO', 'TP_LOCALIZACAO'], observed=True
).agg(agg_dict).reset_index()

In [ ]:
print(f'DF dados_escolas_2020 → {len(dados_escolas_2020):,} registros'.replace(',', '.'))
print(f'DF dados_município_2020 → {len(dados_municipios_2020):,} registros'.replace(',', '.'))

DF dados_escolas_2020 → 28.933 registros
DF dados_município_2020 → 7.066 registros


#### Tratamento dos dados

Ajuste de tipos  

In [ ]:
 # Casting de int64 float32
num = [
    'IN_ATIVA', 'IN_INTERNET', 'IN_ALIMENTACAO', 'REDE_PUBLICA',  'REDE_PRIVADA',  'AR_NAO_DIFER',  'AR_ASSENTAMENTO',
    'AR_INDIGENA',  'AR_REM_QUILOMBO'
]
dados_municipios_2020[num] = dados_municipios_2020[num].astype('float32')

In [ ]:
 # Casting de float64 float32
num = [
    'IN_PARALISADA', 'IN_MUNICIPAL', 'IN_ESTADUAL', 'IN_FEDERAL', 'IN_PRIVADA', 'IN_MED', 'IN_MEDIACAO_PRESENCIAL',
    'IN_MEDIACAO_SEMIPRESENCIAL', 'IN_MEDIACAO_EAD', 'IN_DIURNO', 'IN_NOTURNO', 'QT_MAT_MED'
]
dados_municipios_2020[num] = dados_municipios_2020[num].astype('float32')

#### Inspeção do DataFrame

In [ ]:
validador_nulos = (
    dados_municipios_2020
    .isna()
    .sum()
    .to_frame('Valores Nulos')
    .assign(Percentual=lambda df: (df['Valores Nulos'] / len(dados_municipios_2020) * 100).round(2))
    .sort_values(by='Valores Nulos', ascending=False)
)
print(validador_nulos[validador_nulos['Valores Nulos'] > 0])

                   Valores Nulos  Percentual
TAP_MED3                     328        4.64
TRE_MED3                     328        4.64
TAB_MED3                     328        4.64
TAP_MED2                      82        1.16
TRE_MED2                      82        1.16
TAB_MED2                      82        1.16
ALUNOS_TURMA_MED3             78        1.10
ALUNOS_TURMA_MED2             70        0.99
TAB_MED1                      45        0.64
TRE_MED1                      45        0.64
TAP_MED1                      45        0.64
ALUNOS_TURMA_MED1             34        0.48
TAP_MED                       11        0.16
TAB_MED                       11        0.16
TRE_MED                       11        0.16


### » dados_municipios_loc_2020    



#### Criação do DataFrame  

> Este DataFrame é a junção de dados de outros dois DataFrames: dados_municipios_2020 e Taxa de Transição/Evasão (tx_transição_2020).  
>
> Antes do merge, foi necessário filtar o DF tx_transição da seguinte forma:  
> - Para a variável TP_LOCALIZACAO, foram filtrados  apenas valores 'Urbana e 'Rural'. 'Total' não faz sentido aqui, pois as bases precisam ser compatilizadas de forma desagregada para localização: 'Urbana' ou 'Rural' (não há entrada para um valor chamado 'Total')*.  
>
> - Para a variável TP_DEPENDENCIA, deverá ser usado o valor 'Total', que representa todas as escolas do município, sem distinguir rede pública ou privada, o que está totalmente de acordo com o que representa cada registro do DataFrame dados_municipios_2020, que será 'mergeado' a seguir.     
>
> (*)  Vale citar que o valor 'Total' também não faria sentido, pois não se trata de uma simples média aritmética entre escolas públicas e privadas ou em localização urbana ou rural, mas sim de uma média poderada, que depende da quantidade de alunos por rede e por localização, dados estes que não temos disponíveis. Inclusive, esta coluna TP_LOCALIZAÇÃO não será utilizada na sequência (há outras colunas similares).

In [ ]:
# Filtra o tx_transicao para pegar só Urbana e Rural e dependência Total
tx_transicao_filtrado = tx_transicao_2020[
    (tx_transicao_2020['TP_LOCALIZACAO'].isin(['Urbana', 'Rural'])) &
    (tx_transicao_2020['TP_DEPENDENCIA'] == 'Total')
].reset_index(drop=True)

# Faz o merge com os dados de municípios agrupados
dados_municipios_loc_2020 = pd.merge(
    dados_municipios_2020,
    tx_transicao_filtrado,
    on=['CO_MUNICIPIO', 'TP_LOCALIZACAO'],
    how='left'
)

Exclusão da variável TP_DEPENDENCIA que não fará mais sentido neste DataFrame

In [ ]:
dados_municipios_loc_2020 = dados_municipios_loc_2020.drop(columns=['TP_DEPENDENCIA'])

In [ ]:
print(f'DF dados_municipios_2020 → {len(dados_municipios_2020):,} registros'.replace(',', '.'))
print(f'DF tx_transicao_2020 → {len(tx_transicao_2020):,} registros'.replace(',', '.'))
print(f'Resultado do merge → {len(dados_municipios_loc_2020):,} registros'.replace(',', '.'))

DF dados_municipios_2020 → 7.066 registros
DF tx_transicao_2020 → 19.723 registros
Resultado do merge → 7.066 registros


#### Inspeção do DataFrame

In [ ]:
validador_nulos = (
    dados_municipios_loc_2020
    .isna()
    .sum()
    .to_frame('Valores Nulos')
    .assign(Percentual=lambda df: (df['Valores Nulos'] / len(dados_municipios_loc_2020) * 100).round(2))
    .sort_values(by='Valores Nulos', ascending=False)
)
print(validador_nulos[validador_nulos['Valores Nulos'] > 0])

                   Valores Nulos  Percentual
TAB_MED3                     328        4.64
TRE_MED3                     328        4.64
TAP_MED3                     328        4.64
TEV_MED3                     120        1.70
TAB_MED2                      82        1.16
TAP_MED2                      82        1.16
TRE_MED2                      82        1.16
ALUNOS_TURMA_MED3             78        1.10
TEV_MED2                      78        1.10
ALUNOS_TURMA_MED2             70        0.99
TAP_MED1                      45        0.64
TAB_MED1                      45        0.64
TRE_MED1                      45        0.64
TEV_MED1                      44        0.62
ALUNOS_TURMA_MED1             34        0.48
TEV_MED                       22        0.31
TAP_MED                       11        0.16
TAB_MED                       11        0.16
TRE_MED                       11        0.16


Verificação da existência de linhas duplicadas

In [ ]:
duplicatas_completas = dados_municipios_loc_2020.duplicated()
print(f"Quantidade de linhas duplicadas completas: {duplicatas_completas.sum()}")

Quantidade de linhas duplicadas completas: 0


In [ ]:
# Verifica se existem registros repetidos só considerando CO_MUNICIPIO e TP_LOCALIZACAO
duplicatas_municipio_local = dados_municipios_loc_2020.duplicated(subset=['CO_MUNICIPIO', 'TP_LOCALIZACAO'])
print(f"Quantidade de linhas duplicadas por município + localização: {duplicatas_municipio_local.sum()}")

Quantidade de linhas duplicadas por município + localização: 0


#### Salvando a base para análises posteriores

In [ ]:
dados_municipios_loc_2020.to_csv('municipios_tev_2020.csv', encoding='latin1', index=True)

### » dados_municipios_unicos_2020    



#### Criação do DataFrame

> Este DataFrame faz a agregação de linhas do DataFrame e recalcula as agregações do DataFrame dados_municipios_loc_2020

In [ ]:
def media_ponderada(col, peso):
    def func(x):
        valores = x
        pesos = dados_municipios_loc_2020.loc[x.index, peso]
        validos = ~(valores.isna() | pesos.isna())
        valores_validos = valores[validos]
        pesos_validos = pesos[validos]
        soma_pesos = pesos_validos.sum()
        if soma_pesos == 0:
            return np.nan
        return (valores_validos * pesos_validos).sum() / soma_pesos
    return func

# Cria dicionário de agregação
agg_dict_sem_localizacao = {
    'ANO': 'first',
    'NO_REGIAO': 'first',
    'CO_REGIAO': 'first',
    'SG_UF': 'first',
    'CO_UF': 'first',
    'NO_MUNICIPIO': 'first',

    'IN_INTERNET': 'mean',
    'IN_ALIMENTACAO': 'mean',
    'IN_MEDIACAO_PRESENCIAL': 'mean',
    'IN_MEDIACAO_SEMIPRESENCIAL': 'mean',
    'IN_MEDIACAO_EAD': 'mean',
    'IN_DIURNO': 'mean',
    'IN_NOTURNO': 'mean',

    'IN_MED': 'sum',
    'QT_MAT_MED': 'sum',
    'QT_DOC_MED': 'sum',
    'QT_TUR_MED': 'sum',

    'IND_INFRA_BASICA': media_ponderada('IND_INFRA_BASICA', 'QT_MAT_MED'),
    'IND_ACESSIBILIDADE': media_ponderada('IND_ACESSIBILIDADE', 'QT_MAT_MED'),
    'IND_DIVERSIDADE_MATERIAL_PED': media_ponderada('IND_DIVERSIDADE_MATERIAL_PED', 'QT_MAT_MED'),
    'IND_INFRA_EDUCACIONAL': media_ponderada('IND_INFRA_EDUCACIONAL', 'QT_MAT_MED'),

    'QT_DISPOSITIVOS_ALUNOS': 'sum',

    'QT_MAT_MED_FEM': 'sum',
    'QT_MAT_MED_MASC': 'sum',
    'QT_MAT_MED_ND': 'sum',
    'QT_MAT_MED_BRANCA': 'sum',
    'QT_MAT_MED_PRETA': 'sum',
    'QT_MAT_MED_PARDA': 'sum',
    'QT_MAT_MED_AMARELA': 'sum',
    'QT_MAT_MED_INDIGENA': 'sum',

    'ALUNOS_TURMA_MED': 'mean',
    'ALUNOS_TURMA_MED1': 'mean',
    'ALUNOS_TURMA_MED2': 'mean',
    'ALUNOS_TURMA_MED3': 'mean',

    'TAP_MED': media_ponderada('TAP_MED', 'QT_MAT_MED'),
    'TAP_MED1': 'mean',
    'TAP_MED2': 'mean',
    'TAP_MED3': 'mean',

    'TRE_MED': media_ponderada('TRE_MED', 'QT_MAT_MED'),
    'TRE_MED1': 'mean',
    'TRE_MED2': 'mean',
    'TRE_MED3': 'mean',

    'TAB_MED': media_ponderada('TAB_MED', 'QT_MAT_MED'),
    'TAB_MED1': 'mean',
    'TAB_MED2': 'mean',
    'TAB_MED3': 'mean',

    'TEV_MED': media_ponderada('TEV_MED', 'QT_MAT_MED'),
    'TEV_MED1': 'mean',
    'TEV_MED2': 'mean',
    'TEV_MED3': 'mean',

    'IN_MUNICIPAL': 'sum',
    'IN_ESTADUAL': 'sum',
    'IN_FEDERAL': 'sum',
    'IN_PRIVADA': 'sum',
    'IN_ATIVA': 'sum',
    'IN_PARALISADA': 'sum',
    'REDE_PUBLICA': 'sum',
    'REDE_PRIVADA': 'sum',
    'AR_NAO_DIFER': 'sum',
    'AR_ASSENTAMENTO': 'sum',
    'AR_INDIGENA': 'sum',
    'AR_REM_QUILOMBO': 'sum',

    'RAZ_ALUNO_DOC': media_ponderada('RAZ_ALUNO_DOC', 'QT_MAT_MED'),
    'RAZ_ALUNO_TUR': media_ponderada('RAZ_ALUNO_TUR', 'QT_MAT_MED')
}

# Agrega por município, sem distinção rural/urbano
dados_municipios_unicos_2020 = dados_municipios_loc_2020.groupby(
    'CO_MUNICIPIO', observed=False
).agg(agg_dict_sem_localizacao).reset_index()

In [ ]:
print(f'DF dados_escolas_2020 → {len(dados_municipios_loc_2020):,} registros'.replace(',', '.'))
print(f'DF dados_município_2020 → {len(dados_municipios_unicos_2020):,} registros'.replace(',', '.'))

DF dados_escolas_2020 → 7.066 registros
DF dados_município_2020 → 5.561 registros


In [ ]:
validador_nulos = (
   dados_municipios_unicos_2020
    .isna()
    .sum()
    .to_frame('Valores Nulos')
    .assign(Percentual=lambda df: (df['Valores Nulos'] / len(dados_municipios_unicos_2020) * 100).round(2))
    .sort_values(by='Valores Nulos', ascending=False)
)
print(validador_nulos[validador_nulos['Valores Nulos'] > 0])

          Valores Nulos  Percentual
TRE_MED3            170        3.06
TAP_MED3            170        3.06
TAB_MED3            170        3.06
TEV_MED3             18        0.32
TAB_MED2              1        0.02
TRE_MED2              1        0.02
TAP_MED2              1        0.02


#### Salvando a base para análises posteriores

In [ ]:
dados_municipios_unicos_2020.to_csv('municipios_unicos_tev_2020.csv', encoding='latin1', index=True)